In [ ]:
import ast
import math
import torch
import torchvision
import pickle as pkl
import pandas as pd
import numpy as np
import seaborn as sns
from einops import rearrange
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from adjustText import adjust_text
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
from torchmetrics.functional.classification import average_precision

In [ ]:
with open(
    "../model=slow-50-tap/model=slow-50_method=tap-16x8-w-negatives_ext=sw-0.5_return-feats=val_feats.pkl",
    "rb",
) as f:
    data = pkl.load(f)

metadata = pd.read_csv(
    "/home/dl18206/Desktop/phd/data/panaf/PanAfFull/notebooks/data/panaf-seq_metadata.csv"
)

with open(
    "../model=slow-50-tap/model=slow_r50-w-negatives_type=return_val_feats_feats.pkl",
    "rb",
) as f:
    g_data = pkl.load(f)

# Define class names
with open("../data/behaviours.txt", "rb") as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

In [ ]:
data.keys()

In [ ]:
cnts = []
for l in data["labels"]:
    cnts.append(sum(l).item())
data["cnts"] = cnts

In [ ]:
subclips = []
for name, feat, cls_act_seq, global_feat, global_pred in zip(
    data["names"], data["feats"], data["cas"], g_data["feats"], g_data["preds"]
):
    recon_mean = []
    for i, (f, c) in enumerate(zip(feat, cls_act_seq)):
        subclips.append(
            {
                "name": name,
                "feat": f.detach().cpu().numpy(),
                "cas": c.detach().cpu().numpy(),
                "timestep": i,
                "point_type": "local",
            }
        )
        if i == 0:
            recon_mean.append(f.detach().cpu().numpy())
        else:
            recon_mean.append(f.detach().cpu().numpy())

    subclips.append(
        {
            "name": name,
            "feat": global_feat.detach().cpu().numpy(),
            "cas": global_pred.detach().cpu().numpy(),
            "timestep": i + 1,
            "point_type": "global",
        }
    )
    subclips.append(
        {
            "name": name,
            "feat": np.mean(recon_mean, axis=0),
            "cas": global_pred.detach().cpu().numpy(),
            "timestep": i + 2,
            "point_type": "reconstructed",
        }
    )
df = pd.DataFrame(subclips, columns=["name", "feat", "cas", "timestep", "point_type"])
df = df.merge(metadata, how="left", left_on="name", right_on="subject_id")

**Original results**

In [94]:
per_class_ap = average_precision(
    data["preds"], data["labels"], num_labels=14, task="multilabel", average=None
)
map = average_precision(
    data["preds"], data["labels"], num_labels=14, task="multilabel", average="macro"
)
print(f"mAP: {map}\n{per_class_ap}")

mAP: 0.5010252594947815
tensor([0.4096, 0.5366, 0.3756, 0.7521, 0.5303, 0.8265, 0.2021, 0.3453, 0.3701,
        0.0093, 0.7053, 0.9089, 0.8660, 0.1766])


**Find timstep with highest avg activations**

In [ ]:
data["cas"] = data["cas"].detach().cpu()

In [ ]:
max_avg_acts = []
for sample in data["cas"]:
    sample = torch.sigmoid(sample)
    avg_act = torch.mean(sample, axis=1)
    max_idx = torch.argmax(avg_act)
    max_avg_acts.append(sample[max_idx])
max_avg_preds = torch.stack(max_avg_acts)

In [ ]:
per_class_ap = average_precision(
    max_avg_preds, data["labels"], num_labels=14, task="multilabel", average=None
)
map = average_precision(
    max_avg_preds, data["labels"], num_labels=14, task="multilabel", average="macro"
)
print(f"mAP: {map}\n{per_class_ap}")

**Max across timesteps**

In [ ]:
def max_activation_per_class(logits):
    # logits shape: [T, NUM_CLS]
    max_activations = torch.max(logits, dim=0).values  # Shape: [NUM_CLS]
    return max_activations  # Shape: [1, NUM_CLS]

In [ ]:
max_acts = []
for sample in data["cas"]:
    sample = torch.sigmoid(sample)
    max_acts.append(max_activation_per_class(sample))
max_preds = torch.stack(max_acts)

In [ ]:
per_class_ap = average_precision(
    max_preds, data["labels"], num_labels=14, task="multilabel", average=None
)
map = average_precision(
    max_preds, data["labels"], num_labels=14, task="multilabel", average="macro"
)
print(f"mAP: {map}\n{per_class_ap}")

In [78]:
df = pd.DataFrame(
    {
        "name": data["names"],
        "cnt": data["cnts"],
        "preds": list(data["preds"].detach().cpu()),
        "labels": list(data["labels"].detach().cpu()),
    }
)

In [91]:
df["ap"] = average_precision(
    torch.stack(list(df.preds.values)),
    torch.stack(list(df.labels.values)),
    num_labels=14,
    task="multilabel",
    average="macro",
)

In [81]:
# Calculate the average precision per cnt of labels
for c in df.cnt.unique():
    map = average_precision(
        torch.stack(list(df[df.cnt == c].preds.values)),
        torch.stack(list(df[df.cnt == c].labels.values)),
        num_labels=14,
        task="multilabel",
        average="macro",
    )
    print(f"mAP for {c} labels: {map}")

mAP for 3.0 labels: 0.5833555459976196
mAP for 2.0 labels: 0.5850474834442139
mAP for 1.0 labels: 0.5624498128890991
mAP for 4.0 labels: 0.6938210725784302
mAP for 5.0 labels: 0.8325757384300232
mAP for 6.0 labels: 1.0
mAP for 7.0 labels: 1.0


/home/dl18206/anaconda3/envs/dataset-upgrade/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in macro-average
  warnings.warn(*args, **kwargs)


In [87]:
map = average_precision(
    torch.stack(list(df[df.cnt == 6.0].preds.values)),
    torch.stack(list(df[df.cnt == 6.0].labels.values)),
    num_labels=14,
    task="multilabel",
    average=None,
)
print(map)

tensor([1., nan, nan, 1., 1., nan, nan, 1., 1., nan, nan, nan, nan, 1.])


In [90]:
torch.sigmoid(torch.stack(list(df[df.cnt == 6.0].preds.values)))

tensor([[3.0594e-03, 1.7753e-05, 1.3627e-02, 9.9994e-01, 3.6761e-04, 3.7921e-05,
         1.0911e-03, 3.5035e-02, 2.3946e-02, 5.4596e-04, 4.4135e-03, 3.4096e-07,
         1.0000e+00, 2.8183e-03]])

In [ ]:
df = pd.DataFrame(
    {
        "original": average_precision(
            data["preds"],
            data["labels"],
            num_labels=14,
            task="multilabel",
            average=None,
        ),
        "max_avg": average_precision(
            max_avg_preds,
            data["labels"],
            num_labels=14,
            task="multilabel",
            average=None,
        ),
        "max": average_precision(
            max_preds, data["labels"], num_labels=14, task="multilabel", average=None
        ),
    }
)

In [ ]:
df

In [ ]:
df.plot.bar(figsize=(10, 5))
# Plot behaviours on x axis
plt.xticks(range(14), behaviours, rotation=45, ha="right")
plt.ylabel("Average Precision")
plt.xlabel("Behaviour")
plt.title("Average Precision per Behaviour")
plt.show()

In [ ]:
few_shot_classes = ["aggression", "display", "piloerection", "playing"]
few_shot_indices = [behaviours.index(beh) for beh in few_shot_classes]

tail_classes = ["tool_use", "object_carrying", "bipedal", "grooming", "vocalisation"]
tail_indices = [behaviours.index(beh) for beh in tail_classes]

middle_classes = ["feeding", "climbing", "camera_reaction"]
middle_indices = [behaviours.index(beh) for beh in middle_classes]

head_classes = ["travel", "resting"]
head_indices = [behaviours.index(beh) for beh in head_classes]

# Calculate average precision for few-shot classes
print("Few-shot classes\n", df.loc[few_shot_indices].mean())

# Calculate average precision for tail classes
print("Tail classes\n", df.loc[tail_indices].mean())

# Calculate average precision for middle classes
print("Middle classes\n", df.loc[middle_indices].mean())

# Calculate average precision for head classes
print("Head classes\n", df.loc[head_indices].mean())